# 02c Iteradores, generadores, lambdas

## Comprensión de listas
Uno de los usos más comunes de un bucle `for` es almacenar valores en una lista:

In [1]:
cubos = []
for i in range(8):
    cubos.append(i**3)

print(cubos)

[0, 1, 8, 27, 64, 125, 216, 343]


Este mismo código se puede escribir de forma más elegante, en una sola línea, usando una comprensión de lista:

In [2]:
cubos = [i**3 for i in range(8)]

print(cubos)

[0, 1, 8, 27, 64, 125, 216, 343]


Se pueden filtrar elementos:

In [3]:
cubos_impares = [i**3 for i in range(8) if i%2 == 1]

print(cubos_impares)

[1, 27, 125, 343]


que sería equivalente al bucle

In [4]:
cubos_impares = []

for i in range(8):
    if i%2 == 1:
        cubos_impares.append(i**3)

print(cubos_impares)

[1, 27, 125, 343]


Además de comprensión de listas, también existen comprensión de conjuntos y de diccionarios, que funcionan de un modo similar:

In [6]:
letras = {letra for letra in 'palabra'}
print(letras)

{'p', 'b', 'l', 'a', 'r'}


In [7]:
cubos_impares = {i: i**3 for i in range(8) if i%2 == 1}
print(cubos_impares)

{1: 1, 3: 27, 5: 125, 7: 343}


y también podemos usar una comprensión con la función `sum`:

In [77]:
sum(i**3 for i in range(8) if i%2 == 1)

496

## Iteradores

Un iterador es cualquier objeto que se pueda usar como `in` en un bucle `for`. Algunos tipos definidos por Python, como `list`, `tuple`, `set` y `str` son iteradores:

In [8]:
for letra in 'palabra':
    print(letra)

p
a
l
a
b
r
a


Otro iterador muy común es `range()`. En Python 3, `range()` no genera una lista de valores, sino que los genera uno a uno. Para generarlos todos de una vez, tenemos que convertir el `range` en una lista:

In [10]:
print(range(7))

print(list(range(7)))

range(0, 7)
[0, 1, 2, 3, 4, 5, 6]


En general, un iterador es cualquier objeto que tenga definido un método `__next__()`. Al iniciar cada paso del bucle, se llama a `__next__()`, y su valor devuelto se guarda en la variable del bucle. También debe tener un método `__iter__()` que devuelva un objeto iterable, que por lo general es `self`:

In [18]:
from time import sleep

class range_lento:
    def __init__(self):
        self.valor = 0

    def __next__(self):
        sleep(2.5)
        self.valor += 1
        return self.valor

    def __iter__(self):
        return self

In [17]:
for i in range_lento():
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

¡Hemos creado un iterador infinito! Para señalar al bucle `for` cuando parar, hay que lanzar una excepción `StopIteration`:

In [52]:
class range_lento:
    def __init__(self, max):
        self.valor = 0
        self.max = max

    def __next__(self):
        if self.valor <= self.max:
            sleep(2.5)
            self.valor += 1
            return self.valor
        else:
            raise StopIteration

    def __iter__(self):
        return self

In [53]:
for i in range_lento(4):
    print(i)

1


In [50]:
class FloatRange:
    def __init__(self, start, stop, step=1.0):
        if start >= stop:
            raise ValueError("Invalid range")
        self.start = start
        self.stop = stop
        self.step = step

    def __iter__(self):
        n = self.start
        while n < self.stop:
            yield n
            n += self.step

    def __reversed__(self):
        n = self.stop - self.step
        while n >= self.start:
            yield n
            n -= self.step

In [51]:
for number in FloatRange(0.0, 5.0, 0.5):
    print(number)

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


<a id='for'></a>Un bucle `for` en realidad lo que hace es llamar a `next` hasta que se encuentra una excepción `StopIteration`. Este código:

```python
for x in range(5):
    print(x)
```

es equivalente a este otro:

```python
_iter_range = iter(range(5))
while 1:
    try:
        x = next(_iter_range)
    except StopIteration:
        break
    print(x)
```

## Generadores

Un generador es un tipo especial de fuunción que crea un iterador.

En una función ordinaria, una vez que se alcanza `return` (o `raise`), se sale de la función y python olvida completamnete el estado de sus variables internas, de modo que la siguiente vez que se llama a la función, se empieza desde el principio.

Un generador tiene una o más expresiones `yield`. Cuando se alcanza `yield`, se sale del generador, pero se conserva su estado interno. La próxima vez que se llama al generador, se empieza desde la línea siguiente al `yield`, conservando los valores de las variables internas.

In [23]:
def gen():
    x = 2+3
    yield 'Hola'
    yield x

for i in gen():
    print(i)

Hola
5


Un generador puede combinar varios `yield` y un `return`. Cuando se alcance el `return`, se acaba la ejecución del iterador:

In [24]:
def gen():
    x = 2+3
    yield 'Hola'
    yield x
    return 7
    yield 'a'

for i in gen():
    print(i)

Hola
5


Se pueden crear generadores con una sintaxis similar a la de una comprensión de lista, pero encerrados en paréntesis en vez de corchetes:

In [25]:
for i in (i**3 for i in range(8)):
    print(i)

0
1
8
27
64
125
216
343


Puede parecer similar a una comprensión de lista, pero la diferencia es que solamente se evalúa a cada paso del bucle. Compara estos dos códigos:

In [26]:
def cubo(x):
    sleep(2)
    return x**3

In [30]:
miscubos = (cubo(i) for i in range(8))
print("Generador creado")

for i in miscubos:
    print(i)

Generador creado
0
1
8
27
64
125
216
343


In [31]:
miscubos = [cubo(i) for i in range(8)]
print("Lista creada")


for i in miscubos:
    print(i)

Lista creada
0
1
8
27
64
125
216
343


## Manipulación de iteradores

El paquete `random` de la librería estándar contiene algunas funciones para trabajar con secuencias. Una secuencia es un objeto con una longitud determinada (implementa `__len__()`) y cuyos elementos se pueden obtener mediante índices numéricos (implementa `__getitem__()`). Las listas, tuplas y `range` son ejemplos de secuencias.

In [42]:
import random

# Elige un número aleatorio entre 15 y 20
random.choice(range(15, 20))

15

In [46]:
# Reordena los elementos
l = ['a', 'b', 'c', 'd']
random.shuffle(l)
print(l)

['d', 'c', 'a', 'b']


In [47]:
# Elige 4 elementos aleatoriamente (sin repetición)
random.sample(range(20), 4)

[13, 18, 8, 3]

In [48]:
# Elige 20 elementos aleatoriamente (con repetición)
random.choices(range(5), k=20)

[2, 2, 0, 4, 0, 3, 0, 4, 0, 3, 3, 0, 3, 1, 0, 3, 2, 3, 1, 3]

Python tiene una función para darle la vuelta a un iterador, `reversed`. Se aplica a las secuencias, o a los iteradores que implementen `__reveresed__()`:

In [54]:
for i in reversed(range(10)):
    print(i)

9
8
7
6
5
4
3
2
1
0


Vamos a crear nuestro propio iterador con `__reversed__()`:

In [66]:
class range_lento:
    def __init__(self, max):
        self.max = max
    
    def __iter__(self):
        i = 0
        while i < self.max:
            sleep(1.2)
            yield i
            i += 1

    def __reversed__(self):
        i = self.max - 1
        while i >= 0:
            sleep(1.2)
            yield i
            i -= 1

In [59]:
for i in reversed(range_lento(5)):
    print(i)

4
3
2
1
0


En el caso de las listas, es posible recorrerlas en sentido inverso sin necesitar `reversed`. Además de ser indexables desde el principio, con `x[0]`, las listas de Python también son indexables desde el final: `x[-1]` corresponde al último valor, `x[-2]` al penúltimo, y así sucesivamente:

In [91]:
x = [0, 1, 2, 3, 4, 5]

for i in range(-1, -len(x)-1, -1):
    print(x[i])

5
4
3
2
1
0


Las listas también permiten obtener una "rodaja" (slice), con la notación `x[inicio:fin:paso]`. Si `inicio` no se especifica, el valor por defecto es 0, si `fin` no se especifica, el valor por defecto es `len(x)`, y si `paso` no se especifica, el valor por defecto es 1. Si el valor de `paso` es -1, la lista se invierte:

In [92]:
x = [0, 1, 2, 3, 4, 5]
x[::-1]

[5, 4, 3, 2, 1, 0]

Otra función de python para manipular iteradores es `sorted`. Esta función produce una lista con todos los elementos producidos por el iterador, ordenados de menor a mayor. Si es necesario, calcula todos los elementos.

In [67]:
sorted(reversed(range_lento(5)))

[0, 1, 2, 3, 4]

Si no se pasa ningún argumento más, `sorted` compara los elementos del iterador entre sí (para valores numéricos por su valor, para texto según sus códigos ascii/unicode, etc). En concreto, significa que, cuando ordenamos palabras, las que empiecen por mayúscula aparecerán antes que las que empiezan por minúscula, independientemente del orden alfabético:

In [68]:
sorted(['ayer', 'Hoy', 'mañana'])

['Hoy', 'ayer', 'mañana']

Para evitarlo, hay que pasar a `sorted` un argumento `key` que se corresponda con la función usada para ordenar los elementos. En este caso, usaremos `str.lower`, que convierte un string a minúsculas:

In [69]:
sorted(['ayer', 'Hoy', 'mañana'], key=str.lower)

['ayer', 'Hoy', 'mañana']

Como otro ejemplo, vamos a ordenar las palabras por el número de veces que aparezca la letra 'a':

In [71]:
def numero_a(s):
    return len([letra for letra in s.lower() if letra == 'a'])

sorted(['ayer', 'Hoy', 'mañana'], key=numero_a)

['Hoy', 'ayer', 'mañana']

## Expresiones lambda

En el último ejemplo, hemos definido una función `numero_a()`, cuyo cuerpo es una expresión `return`, y que solo necesitamos una vez en el código. Una alternativa en estos casos es usar, en vez de una función, una expresión lambda o "función anónima":

In [72]:
sorted(['ayer', 'Hoy', 'mañana'], key=lambda s: len([letra for letra in s.lower() if letra == 'a']) )

['Hoy', 'ayer', 'mañana']

La sintaxis es la siguiente: primero el identificador `lambda`, después una lista de los argumentos, separados por comas si hay más de uno, después `:`, y finalmente el valor devuelto.

Las lambdas son ejecutables, y se les llama como a cualquier otra función:

In [73]:
(lambda x, y: x+y)(2, 3)

5

Las lambdas son objetos, así que pueden ser asignadas a una variable (aunque el manual de estilo de python lo desaconseja, y recomienda usar en estos casos una función normal):

In [74]:
suma = lambda x, y: x+y
suma(2, 3)

5

Otro caso en el que las expresiones lambda se suelen usar junto a iteradores son los filtros. Un filtro devuelve solamente los elementos de un iterador que cumplen una condición:

In [76]:
for i in filter(lambda x: len(x) > 3, ['ayer', 'Hoy', 'mañana']):
    print(i)

ayer
mañana


## Algunos iteradores comunes

Un archivo abierto con `open` en modo `rt` actúa como un generador sobre sus líneas:

In [2]:
with open("input_02c", "rt") as f:
    for linea in f:
        print(linea.strip())

1,8,9
5,11,3
11,7,16
13,14,4
11,3,10
15,15,13
13,5,9
9,14,3
2,11,11
6,13,15
3,12,11
8,12,17
2,9,8
12,8,15
5,11,6
8,2,12
9,5,2
12,13,15
10,15,7
14,12,3
10,2,6
14,5,15
16,11,9
13,15,14
3,4,12
3,8,6
8,8,1
12,3,8
12,14,5
3,11,12
16,14,9
16,5,14
7,6,5
3,6,5
16,12,9
4,13,12
7,16,9
1,11,10
11,4,14
5,16,7
11,15,7
9,15,14
10,16,7
13,3,14
10,1,8
17,9,6
6,3,10
3,5,9
6,12,3
12,6,16
7,12,4
13,12,16
14,3,9
11,8,3
17,8,11
7,13,2
3,6,8
14,15,10
8,6,3
6,2,9
10,17,10
7,12,3
12,16,5
2,12,8
15,15,11
10,18,11
9,7,17
4,12,14
9,18,9
12,13,17
14,4,14
17,6,12
17,11,6
2,11,7
13,17,9
7,3,9
8,5,4
14,7,16
14,11,17
10,3,16
5,6,4
13,3,12
14,15,7
13,10,15
14,16,7
12,15,5
15,11,7
12,5,3
15,8,5
15,9,15
3,14,12
7,11,1
7,14,5
3,5,8
14,3,7
8,4,14
3,8,8
16,8,10
14,16,9
6,6,3
12,13,2
12,10,3
9,2,12
11,15,6
13,17,11
15,16,9
14,10,16
4,11,5
12,15,13
4,5,12
10,2,8
15,14,7
15,5,14
9,17,8
9,5,16
5,14,14
8,9,16
3,11,9
9,8,3
11,12,3
8,5,3
3,10,5
17,8,14
15,5,6
8,4,5
2,11,5
2,10,10
14,5,14
16,6,5
9,16,13
4,8,17
16,10,14
3,7,10
12,6

El iterador `zip` combina elementos de dos o más iteradores en una tupla:

In [78]:
for z in zip(['a', 'b', 'c'], 'ABC'):
    print(z)

('a', 'A')
('b', 'B')
('c', 'C')


En el caso de que los dos iteradores tengan distinta longitud, el `zip` termina cuando lo hace el más corto:

In [79]:
for z in zip(['a', 'b', 'c'], 'ABCDEF'):
    print(z)

('a', 'A')
('b', 'B')
('c', 'C')


Un diccionario ofrece tres iteradores: `.keys()` para iterar por sus claves, `.values()` por sus valores, e `.items()`, que es un `zip` de los dos anteriores:

In [80]:
d = {'a': 1, 'b': 2, 'c': 3}

print("Claves:")
for k in d.keys():
    print(k)

print("\nValores:")
for v in d.values():
    print(v)

print("\nItems:")
for k, v in d.items():
    print(f"{k}: {v}")

Claves:
a
b
c

Valores:
1
2
3

Items:
a: 1
b: 2
c: 3


El iterador `enumerate` es equivalente a un `zip` en el que el primer elemento es un `range()`:

In [81]:
for i, letra in enumerate('palabra'):
    print(f"La letra en la posición {i} es {letra}")

La letra en la posición 0 es p
La letra en la posición 1 es a
La letra en la posición 2 es l
La letra en la posición 3 es a
La letra en la posición 4 es b
La letra en la posición 5 es r
La letra en la posición 6 es a


El iterador `map` aplica una función a cada elemento de un iterador. Es muy similar a los generadores que hemos visto antes:

In [82]:
for i in map(lambda x: x**3, range(8)):
    print(i)

0
1
8
27
64
125
216
343


Hay más iteradores en el paquete `itertools` de la librería estándar. Por ejemplo, `itertools.chain` concatena dos iteradores:

In [84]:
import itertools

for x in itertools.chain('abc', range(7)):
    print(x)

a
b
c
0
1
2
3
4
5
6


`itertools.count()` es un `range` infinito (como el que hemos definido antes):

In [86]:
for i in itertools.count():
    print(i)
    if i > 500:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

`itertools.cycle()` va repitiendo de forma cíclica los elementos indefinidamente:

In [87]:
for i, x in enumerate(itertools.cycle('abcd')):
    print(x)
    if i > 20:
        break

a
b
c
d
a
b
c
d
a
b
c
d
a
b
c
d
a
b
c
d
a
b


`itertools.repeat()` repite un elemento varias veces:

In [98]:
for x in itertools.repeat(2.0, 5):
    print(x)

2.0
2.0
2.0
2.0
2.0


Si en vez de un iterador quieres una lista con elementos repetidos, se puede conseguir con `*`:

In [99]:
for x in [2.0]*5:
    print(x)

2.0
2.0
2.0
2.0
2.0


`itertools.accumulate` produce la suma cumulativa de los elementos:

In [89]:
list(itertools.accumulate([1, 2, 3, 4, 5]))

[1, 3, 6, 10, 15]

Se puede pasar una función a `accumulate`, y usará esta versión en vez de hacer la suma. Por ejemplo, si queremos obtener el máximo cumulativo:

In [95]:
x = [random.random() for i in range(10)]
print(x)
print(list(itertools.accumulate(x, max)))

[0.11744710697957816, 0.5258243932019212, 0.016466888907123223, 0.9952213307861382, 0.16113348592195842, 0.2805280931505578, 0.5370830084883165, 0.3362236234049595, 0.1400167031842221, 0.07205971200844785]
[0.11744710697957816, 0.5258243932019212, 0.5258243932019212, 0.9952213307861382, 0.9952213307861382, 0.9952213307861382, 0.9952213307861382, 0.9952213307861382, 0.9952213307861382, 0.9952213307861382]


`itertools.starmap` funciona de forma similar a `map`, pero para funciones con varios argumentos. Se le tiene que pasar un iterable de secuencias.

Veamos un ejemplo, en el que calculamos $R_K$ en varios bins:

In [100]:
import flavio

RK = lambda q2min, q2max: flavio.sm_prediction('<Rmue>(B+->Kll)', q2min, q2max)

In [101]:
RK(1.1, 6.0)

1.0007790786808297

In [103]:
list(itertools.starmap(RK, [(0.045, 1.1), (1.1, 6.0), (15.0, 16.0)]))

[0.9816497435344065, 1.0007790786808297, 1.0020030170126109]

`itertools.product` crea el producto cartesiano de dos o más iteradores, y es equivalente a bucles anidados:

In [104]:
for i, j in itertools.product(range(3), range(5)):
    print(f"({i}, {j})")

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)


In [105]:
for i in range(3):
    for j in range(5):
        print(f"({i}, {j})")

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)


`itertools.permutations()` genera todas las permutaciones (sin repetición):

In [109]:
for p in itertools.permutations('abcd'):
    print(''.join(p))

abcd
abdc
acbd
acdb
adbc
adcb
bacd
badc
bcad
bcda
bdac
bdca
cabd
cadb
cbad
cbda
cdab
cdba
dabc
dacb
dbac
dbca
dcab
dcba


`itertools.combinations()` genera todas las combinaciones (sin repetición):

In [110]:
for p in itertools.combinations('abcd', 2):
    print(''.join(p))

ab
ac
ad
bc
bd
cd


`itertools.combinations_with_replacement()` genera todas las combinaciones con repetición:

In [111]:
for p in itertools.combinations_with_replacement('abcd', 2):
    print(''.join(p))

aa
ab
ac
ad
bb
bc
bd
cc
cd
dd


`numpy` tiene varios objetos que son versiones mejoradas de `range`: `arange` permite que el incremento sea un número decimal:

In [112]:
import numpy as np

for x in np.arange(0, 2, 0.37):
    print(x)

0.0
0.37
0.74
1.1099999999999999
1.48
1.85


`linspace` permite crear `n` puntos equiespaciados entre el valor inicial y el final:

In [113]:
for x in np.linspace(0, 5, 21):
    print(x)

0.0
0.25
0.5
0.75
1.0
1.25
1.5
1.75
2.0
2.25
2.5
2.75
3.0
3.25
3.5
3.75
4.0
4.25
4.5
4.75
5.0


y `logspace` crea los puntos espaciados logarítmicamente. Hay que especificar la base:

In [114]:
for x in np.logspace(0, 4, 21, base=2):
    print(x)

1.0
1.148698354997035
1.3195079107728942
1.5157165665103982
1.7411011265922482
2.0
2.29739670999407
2.639015821545789
3.0314331330207964
3.4822022531844965
4.0
4.59479341998814
5.278031643091579
6.062866266041593
6.964404506368995
8.0
9.18958683997628
10.556063286183157
12.125732532083186
13.92880901273799
16.0


In [115]:
for x in np.linspace(0, 4, 21):
    print(2**x)

1.0
1.148698354997035
1.3195079107728942
1.5157165665103982
1.7411011265922482
2.0
2.29739670999407
2.639015821545789
3.0314331330207964
3.4822022531844965
4.0
4.59479341998814
5.278031643091579
6.062866266041593
6.964404506368995
8.0
9.18958683997628
10.556063286183157
12.125732532083186
13.92880901273799
16.0


## Tuberías

En los casos en los que hay que procesar un gran número de datos, una de las técnicas comunes es el uso de "tuberías" (pipelines), formadas por generadores en serie. Cada generador toma datos del generador anterior y lo procesa.

Vamos a crear una tubería en la cual leemos el archivo `input_02c`, en cada línea separamos las tres componentes de un vector y las convertimos a tipo `int`, a continuación calculamos el módulo del vector, seleccionamos cuántos de ellos tienen un módulo menor de 10, y los contamos.

In [3]:
with open("input_02c", "rt") as f:
    separados = (line.strip().split(",") for line in f)
    vectores = ((int(s[0]), int(s[1]), int(s[2])) for s in separados)
    modulos = ((v[0]**2+v[1]**2+v[2]**2)**0.5 for v in vectores)
    modulos_menor10 = (m for m in modulos if m < 10)
    numero = sum(1 for m in modulos_menor10)

print(numero)

98


El archivo `f` actúa como fuente de la tubería, y la función `sum` como el sumidero. Cada elemento solo se procesa cuando `sum` pide un nuevo valor, por lo que no hace falta guardar en memoria todos los vectores o sus módulos. De hecho, si intentamos sacar alguno de los generadores del bloque `with` obtendremos un error, ya que el archivo ya estará cerrado cuando intentamos procesarlo:

In [11]:
with open("input_02c", "rt") as f:
    separados = (line.strip().split(",") for line in f)
vectores = ((int(s[0]), int(s[1]), int(s[2])) for s in separados)
modulos = ((v[0]**2+v[1]**2+v[2]**2)**0.5 for v in vectores)
modulos_menor10 = (m for m in modulos if m < 10)
numero = sum(1 for m in modulos_menor10)

print(numero)

ValueError: I/O operation on closed file.

Podemos crear un sumidero con un código similar al [código interno del bucle `for`](#for). Por ejemplo, para calcular el producto de los elementos de un iterador:

In [4]:
def producto(it):
    _iterable = iter(it)
    p = 1
    while 1:
        try:
            p *= next(_iterable)
        except StopIteration:
            return p

In [6]:
producto(range(1, 7)) # 6!

720

In [9]:
with open("input_02c", "rt") as f:
    separados = (line.strip().split(",") for line in f)
    vectores = ((int(s[0]), int(s[1]), int(s[2])) for s in separados)
    modulos = ((v[0]**2+v[1]**2+v[2]**2)**0.5 for v in vectores)
    modulos_menor10 = (m for m in modulos if m < 10)
    prod = producto(m for m in modulos_menor10)

print(prod)

6.481033952968496e+93
